In [1]:
import data_handler as dh
import visualization as viz
import spatial_features as spf
import game_state_representation as gsr
import pandas as pd

In [2]:
FIELD_DIMEN = (110, 72)
DATA_DIR_single = '../bru_data/data/comp-4zwgbb66rif2spcoeeol2motx/tmcl-1qtpbdbeudho5i7fu5z2lp2j8/fx-4sekj9hgwxzq3y4ih9415239w'
DATA_DIR = '../bru_data/data/comp-4zwgbb66rif2spcoeeol2motx/tmcl-1qtpbdbeudho5i7fu5z2lp2j8'

event_data_single = dh.read_event_data(DATA_DIR_single)
# event_data = dh.read_dir_event_data(DATA_DIR)

In [3]:
event_data_single

,typeId,periodId,timeMin,timeSec,outcome,x,y,qualifier,playerId,playerName,keyPass,assist
0,34,16,0,0,1,0.0,0.0,"[{'id': 3521403699, 'qualifierId': 59, 'value'...",NaN,NaN,NaN,NaN
1,34,16,0,0,1,0.0,0.0,"[{'id': 3521407201, 'qualifierId': 30, 'value'...",NaN,NaN,NaN,NaN
2,32,1,0,0,1,0.0,0.0,"[{'id': 3521547939, 'qualifierId': 127, 'value...",NaN,NaN,NaN,NaN
3,32,1,0,0,1,0.0,0.0,"[{'id': 3521547937, 'qualifierId': 127, 'value...",NaN,NaN,NaN,NaN
4,1,1,0,0,1,50.7,49.1,"[{'id': 3523515873, 'qualifierId': 213, 'value...",x3g0yyxs0j4o8fxphxtf0st1,H. Vanaken,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1744,30,2,93,46,1,0.0,0.0,"[{'id': 3522028543, 'qualifierId': 57, 'value'...",NaN,NaN,NaN,NaN
1745,30,14,0,0,1,0.0,0.0,"[{'id': 3522028863, 'qualifierId': 57, 'value'...",NaN,NaN,NaN,NaN
1746,37,14,0,0,1,0.0,0.0,"[{'id': 3524800621, 'qualifierId': 406, 'value...",NaN,NaN,NaN,NaN
1747,30,14,0,0,1,0.0,0.0,"[{'id': 3522035583, 'qualifierId': 209}, {'id'...",NaN,NaN,NaN,NaN


In [4]:
tracking_single = dh.read_tracking_data(DATA_DIR_single)

In [5]:
tracking_single

,Timestamp,Framecount,Match period,Match status,Column 5,Ball xyz
0,1650807000000,0,1,0,"[{'Object type': '0', 'Player id': '13d7ao0j09...","[52.4, 32.93, 0.0]"
1,1650807000100,100,1,0,"[{'Object type': '0', 'Player id': '13d7ao0j09...","[52.45, 32.665, 0.0]"
2,1650807000200,200,1,0,"[{'Object type': '0', 'Player id': '13d7ao0j09...","[52.5, 32.4, 0.0]"
3,1650807000300,300,1,0,"[{'Object type': '0', 'Player id': '13d7ao0j09...","[52.565, 32.395, 0.0]"
4,1650807000400,400,1,0,"[{'Object type': '0', 'Player id': '13d7ao0j09...","[52.65, 32.57, 0.0]"
...,...,...,...,...,...,...
56268,1650813517600,2917600,2,0,"[{'Object type': '0', 'Player id': '13d7ao0j09...","[10.09, 58.46, 0.0]"
56269,1650813517700,2917700,2,0,"[{'Object type': '0', 'Player id': '13d7ao0j09...","[9.62, 58.315, 0.0]"
56270,1650813517800,2917800,2,0,"[{'Object type': '0', 'Player id': '13d7ao0j09...","[9.17, 58.18, 0.0]"
56271,1650813517900,2917900,2,0,"[{'Object type': '0', 'Player id': '13d7ao0j09...","[8.735, 58.045, 0.0]"


In [6]:
data = dh.read_event_tracking_data(DATA_DIR, FIELD_DIMEN)

Reading file 1...
-- Reading event 4
-- Reading event 5
-- Reading event 8
-- Reading event 11
-- Reading event 12
-- Reading event 13
-- Reading event 16
-- Reading event 17
-- Reading event 19
-- Reading event 21
-- Reading event 25
-- Reading event 26
-- Reading event 28
-- Reading event 29
-- Reading event 35
-- Reading event 38


KeyboardInterrupt: 